In [137]:
import pandas as pd
from geopy.geocoders import Nominatim
import numpy as np
import phonenumbers
from phonenumbers.phonenumberutil import region_code_for_number
import requests
import pycountry
from fuzzywuzzy import process
import random
import string
import regex as re
import datetime


In [31]:
accounts_df = pd.read_excel("Vibely Dataset.xlsx", sheet_name="Accounts")


### Country

In [32]:
country_cleaning = accounts_df[["city", "country", "ip_address", "phone_number"]]

In [33]:
def country_code_to_name(country_code):
    try:
        country = pycountry.countries.get(alpha_2=country_code)
        if country:
            return country.name
        else:
            return np.nan
    except:
        return np.nan

def ip_to_country(ip_address):
        
    try:
        response = requests.get(f"https://ipinfo.io/{ip_address}/json")
        if response.status_code == 200:
            data = response.json()
            return country_code_to_name(data.get('country'))
        else:
            return np.nan
    except:
        return np.nan
    

def city_to_country(city):
    geolocator = Nominatim(user_agent="city_to_country_converter")
    location = geolocator.geocode(city, language="en")
    if location:
        return location.address.split(",")[-1].strip()
    else:
        return np.nan


def phone_number_to_country(phone_number):
    try:
        parsed_number = phonenumbers.parse(phone_number)
        country_code = region_code_for_number(parsed_number)
        if country_code:
            return country_code_to_name(country_code)
        else:
            return np.nan
    except:
        return np.nan


In [34]:
# Non standard country codes exist in data: remove them and then run fill script to fill them
mask = country_cleaning["country"].apply(lambda x: isinstance(x, str) and len(x.strip()) == 2)
country_cleaning.loc[mask, "country"] = np.nan


C:\Users\DomMaciulaitis\AppData\Local\Temp\ipykernel_12972\942570911.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_cleaning.loc[mask, "country"] = np.nan


In [35]:
# Fill missing country values using phone number
mask = country_cleaning["country"].isna() & ~country_cleaning["phone_number"].isna()
country_cleaning.loc[mask, "country"] = country_cleaning.loc[mask, "phone_number"].apply(lambda x: phone_number_to_country(x))

# Fill missing values using the city
mask = country_cleaning["country"].isna() & ~country_cleaning["city"].isna()
country_cleaning.loc[mask, "country"] = country_cleaning.loc[mask, "city"].apply(lambda x: city_to_country(x))

# Fill remaining missing country values using the ip address
mask = country_cleaning["country"].isna() & ~country_cleaning["ip_address"].isna()
country_cleaning.loc[mask, "country"] = country_cleaning.loc[mask, "ip_address"].apply(lambda x: ip_to_country(x))

C:\Users\DomMaciulaitis\AppData\Local\Temp\ipykernel_12972\1536486857.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_cleaning.loc[mask, "country"] = country_cleaning.loc[mask, "phone_number"].apply(lambda x: phone_number_to_country(x))
C:\Users\DomMaciulaitis\AppData\Local\Temp\ipykernel_12972\1536486857.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_cleaning.loc[mask, "country"] = country_cleaning.loc[mask, "city"].apply(lambda x: city_to_country(x))
C:\Users\DomMaciulaitis\AppData\Local\Temp\ipykernel_12972\1536486857.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [36]:
def consistent_country(country_input:str):
    countries = [
        "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia",
        "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium",
        "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria",
        "Burkina Faso", "Burundi", "Cabo Verde", "Cambodia", "Cameroon", "Canada", "Central African Republic", "Chad",
        "Chile", "China", "Colombia", "Comoros", "Congo", "Costa Rica", "Croatia", "Cuba", "Cyprus", "Czech Republic",
        "Denmark", "Djibouti", "Dominica", "Dominican Republic", "East Timor", "Ecuador", "Egypt", "El Salvador",
        "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini", "Ethiopia", "Fiji", "Finland", "France", "Gabon",
        "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea", "Guinea-Bissau", "Guyana",
        "Haiti", "Honduras", "Hungary", "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy",
        "Ivory Coast", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "Kosovo", "Kuwait", "Kyrgyzstan",
        "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg",
        "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", "Mauritius",
        "Mexico", "Micronesia", "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar",
        "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "North Korea",
        "North Macedonia", "Norway", "Oman", "Pakistan", "Palau", "Palestine", "Panama", "Papua New Guinea", "Paraguay",
        "Peru", "Philippines", "Poland", "Portugal", "Qatar", "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis",
        "Saint Lucia", "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia",
        "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", "Solomon Islands",
        "Somalia", "South Africa", "South Korea", "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Sweden",
        "Switzerland", "Syria", "Taiwan", "Tajikistan", "Tanzania", "Thailand", "Togo", "Tonga", "Trinidad and Tobago",
        "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom",
        "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Vatican City", "Venezuela", "Vietnam", "Yemen", "Zambia",
        "Zimbabwe"
    ]

    if isinstance(country_input, str) and country_input:
        return process.extractOne(country_input, countries)[0]
    else:
        return "Prefer not to say"
        

In [37]:
# Correcting inconsistent country inputs using fuzzy matching
mask = ~country_cleaning["country"].isna()
accounts_df.loc[mask, "country"] = country_cleaning.loc[mask, "country"].apply(lambda x: consistent_country(x))
 

### Consistent gender

In [38]:
def consistent_gender(gender_input:str):
    valid_set = ["Male", "Female", "Gender Fluid", "Non Binary", "Prefer not to say"]

    if isinstance(gender_input, str) and gender_input:
        if gender_input == "M":
            return "Male"
        elif gender_input == "F":
            return "Female"
        else:
            return process.extractOne(gender_input, valid_set)[0]
    else:
        return "Prefer not to say"
        
    

In [39]:
accounts_df.loc[:, "gender"] = accounts_df.loc[:, "gender"].apply(lambda x: consistent_gender(x))

### Usernames

In [40]:
def username_from_email(email):
    at_index = email.find("@")
    return email[:at_index]

In [41]:
mask = accounts_df["username"].isna() & ~accounts_df["email"].isna()
accounts_df.loc[mask, "username"] = accounts_df.loc[mask, "email"].apply(lambda x: username_from_email(x))

In [42]:
def generate_random_username(length=8):
    characters = string.ascii_lowercase + string.digits
    return ''.join(random.choice(characters) for i in range(length))

existing_users = accounts_df["username"].tolist()
for index, row in accounts_df.iterrows():
    
    if pd.isna(row["username"]):
        random_username = generate_random_username()
        if random_username not in existing_users:
            accounts_df.iloc[index, 2] = random_username
            existing_users.append(random_username)


#### Merge and deduplicate users

In [43]:
# Add a column to indicate duplicates
accounts_df['is_duplicate'] = (~accounts_df['username'].isnull()) & accounts_df.duplicated(subset='username', keep=False)

# Function to combine interests into a list
def combine_interests(group):
    all_interests = ','.join(group['interests']).split(',')
    unique_interests = list(set(all_interests))
    return pd.Series({'combined_interests': [','.join(unique_interests)]})


# Group by 'username' and aggregate 'interests' into a list
merged_df = accounts_df.groupby(['username', 'is_duplicate']).apply(combine_interests)
merged_df.reset_index(inplace=True)

accounts_df.drop(columns=["is_duplicate"], inplace = True)
semi_clean_df = accounts_df.merge(merged_df[["combined_interests", "username"]], on='username', how='left')


In [44]:
semi_clean_df = accounts_df.merge(merged_df[["combined_interests", "username"]], on='username', how='left')
semi_clean_df.drop_duplicates(subset=['username'], inplace=True)

### Subscription

In [45]:
# Continue with semi_clean_df
semi_clean_df.head()

,first_name,interests,username,email,last_name,city,country,gender,password,birth_date2,card_number,job_title,ip_address,birth_date,subscription,account_creation_date,card_type,phone_number,profile_picture_URL,combined_interests
0,Hirsch,"Environment, Philosophy, Culture",hbodley0,hbodley0@slashdot.org,Bodley,Bosilovo,North Macedonia,Male,nW0UuPAl,Rev,3.573452e+15,Internal Auditor,74.141.159.245,1950-02-07 00:00:00,0,2021-06-08,jcb,+389 750 613 8778,http://dummyimage.com/133x100.png/5fa2dd/ffffff,"[ Philosophy,Environment, Culture]"
1,Gerri,"Technology, Relationships",gpetricek1,gpetricek1@deliciousdays.com,Petricek,Aizkraukle,Latvia,Female,RQhxIkB0PyV,NaN,3.565316e+15,Chemical Engineer,231.171.32.149,1984-04-18 00:00:00,0,2021-06-11,jcb,+371 110 603 6635,http://dummyimage.com/188x100.png/dddddd/000000,"[ Relationships,Technology]"
2,Lance,"Technology, Philosophy, Transportation",llebel2,liiannone2@rediff.com,NaN,Czarna,Poland,Male,XserTbnunEH,Dr,3.746221e+14,NaN,253.106.140.96,1982-07-14 00:00:00,0,2020-10-11,americanexpress,+48 549 133 0446,http://dummyimage.com/109x100.png/dddddd/000000,"[ Philosophy, Transportation,Technology]"
3,Townie,"Education, Healthcare, Gaming",tbloan3,tbloan3@taobao.com,Bloan,Landim,Portugal,Non Binary,DzdJYy,Rev,3.552007e+15,NaN,124.80.236.185,1947-11-09 00:00:00,1,2019-12-27,jcb,+351 500 434 4711,http://dummyimage.com/101x100.png/dddddd/000000,"[ Healthcare, Gaming,Education]"
4,Chrissy,"Technology, Environment, Philosophy",clyenyng4,clyenyng4@baidu.com,Lyenyng,Hrib-Loški Potok,Slovenia,Female,Gh64XZ,Mrs,3.030084e+13,Biostatistician IV,23.181.127.21,2004-02-09 00:00:00,0,2018-12-05,diners-club,+386 637 266 3033,http://dummyimage.com/189x100.png/5fa2dd/ffffff,"[ Philosophy, Environment,Technology]"


In [46]:
semi_clean_df["subscription"].unique()

array([0, 1, 'No', 'Yes', datetime.datetime(2006, 12, 26, 0, 0), 'O',
       datetime.datetime(1981, 6, 14, 0, 0), 'yes', 'no',
       datetime.datetime(1964, 6, 27, 0, 0),
       datetime.datetime(1962, 10, 17, 0, 0), 'Next month',
       datetime.datetime(1989, 4, 29, 0, 0),
       datetime.datetime(1992, 6, 10, 0, 0), nan, 'Not sure', 'o', 3],
      dtype=object)

In [47]:
def clean_subscription(row):
    if row in [1, "Yes", "yes", "1"]:
        return 1
    else:
        return 0

In [48]:
semi_clean_df.loc[:, "subscription"] = semi_clean_df.loc[:, "subscription"].apply(lambda x: clean_subscription(x))

C:\Users\DomMaciulaitis\AppData\Local\Temp\ipykernel_12972\1136445598.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  semi_clean_df.loc[:, "subscription"] = semi_clean_df.loc[:, "subscription"].apply(lambda x: clean_subscription(x))


### Credit card type

In [79]:
semi_clean_df.head(1)

,first_name,interests,username,email,last_name,city,country,gender,password,birth_date2,card_number,job_title,ip_address,birth_date,subscription,account_creation_date,card_type,phone_number,profile_picture_URL,combined_interests
0,Hirsch,"Environment, Philosophy, Culture",hbodley0,hbodley0@slashdot.org,Bodley,Bosilovo,North Macedonia,Male,nW0UuPAl,Rev,3.573452e+15,Internal Auditor,74.141.159.245,1950-02-07 00:00:00,0,2021-06-08,jcb,+389 750 613 8778,http://dummyimage.com/133x100.png/5fa2dd/ffffff,"[ Culture,Environment, Philosophy]"


In [49]:
semi_clean_df["card_type"].unique()

array(['jcb', 'americanexpress', 'diners-club', 'visa', 'bankcard',
       'debit', 'solo', 'maestro', 'laser', 'switch', 'mastercard',
       'instapayment', 'china-unionpay', 'debit ', 'v1sa', 'credit card ',
       'credit', 'debit-card', 'card', 'instapaym3nt', 'maestrocard ',
       'none', '-', 'bankc@rd', '!!', 'sw!tch', 'maestrocard',
       'credit-card', 'test', 's0lo', 'X', 'paypal', nan, 'd!ners-club',
       'Credit-Card'], dtype=object)

In [56]:
def consistent_card_type(card_type:str):
    valid_set = ["mastercard", "jcb", "americanexpress",
                    "visa", "maestro", "paypal", "dinersclub",
                    "chinaunionpay", "creditcard", "debitcard", "laser",
                    "bankcard", "instapayment", "solo", "switch", "card"
                ]

    if isinstance(card_type, str) and card_type:
        result, threshold = process.extractOne(card_type, valid_set)
        if threshold >= 75:
            return result
        else:
            return np.nan
    else: 
        return np.nan
        
    

In [63]:
semi_clean_df.loc[:, "card_type"] = semi_clean_df.loc[:, "card_type"].apply(lambda x: consistent_card_type(x))

### Birth Date

In [72]:
semi_clean_df["birth_date "]

0        1950-02-07 00:00:00
1        1984-04-18 00:00:00
2        1982-07-14 00:00:00
3        1947-11-09 00:00:00
4        2004-02-09 00:00:00
                ...         
11005    2007-11-08 00:00:00
11006                    NaN
11007    1964-06-05 00:00:00
11008                    NaN
11009                    NaN
Name: birth_date , Length: 11004, dtype: object

In [148]:
def month_fuzzy_match(month_str):
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    months_dict = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}
    return months_dict[process.extractOne(month_str, months)[0]]


def two_len_year(year):
    if len(year) == 2 and int(year) > 23:
        return "19" + year
    elif len(year) == 2:
        return "20" + year
            

def date_list_to_datetime(date):
    a = date[0]
    b = date[1]
    c = date[2]
    if len(c) == 2:
        c = two_len_year(c)
    try:
        return datetime.datetime(day=int(a), month=int(b), year=int(c))
    except:
        return datetime.datetime(day=int(b), month=int(a), year=int(c))


def clean_birth_date(birth_date):
    if isinstance(birth_date, str):
        if "/" in birth_date:
            try:
                date = birth_date.split("/")
                return date_list_to_datetime(date)
            except:
                return np.nan
            
        elif "-" in birth_date:
            try:
                date = birth_date.split("-")
                return date_list_to_datetime(date)
            except:
                return np.nan
            
        else:
            matches = re.search(r"(\d*)?(th)? ?(\w*) (\d*)", birth_date)
            try:
                day, _, month, year = matches.groups()
                if len(year) == 2:
                    year = two_len_year(year)

                if not day:
                    day = 1
                return datetime.datetime(day=int(day), month=month_fuzzy_match(month), year=int(year))
            except:
                return np.nan
    else:
        if isinstance(birth_date, datetime.datetime):
            return birth_date
        else:
            return np.nan

In [149]:
semi_clean_df.loc[:, "birth_date "] = semi_clean_df.loc[:, "birth_date "].apply(lambda x: clean_birth_date(x))

In [150]:
final_df = semi_clean_df

In [155]:
isinstance(final_df["birth_date "], datetime.datetime)

False